In [ ]:
# default_exp stockdata

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# nbdev function - ensures that changed libraries from the project are reloaded
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Stock

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set, Dict

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from yahoo_historical import Fetcher
import yfinance as yf

In [ ]:
all_data_local_folder = "./data/"
stock_data_folder = "D:/data/stocks/sec/"

join_group = ["cik","ticker","adsh","period","filed","form","fp"]

## 00_CIK-Ticker Corrections

After checking the data in a later step, I discovered that some of the CIK-Ticker matching were wrong. In this Step, we correct that.

Corrected Mappings:
- AAWW -> 0001135185
- RSPI -> 849636
- MLSS -> 855683
- NOG  -> 1104485
- PAR -> 708821
- ARMP -> 921114

In [ ]:
df_cik_ticker_old = pd.read_csv(all_data_local_folder + "cik_ticker.csv", sep="|", header = 0)
print(df_cik_ticker_old.shape)

df_cik_ticker_new = df_cik_ticker_old.copy()

df_cik_ticker_new.loc[df_cik_ticker_new.CIK == 1135185, 'Ticker'] = "AAWW"
df_cik_ticker_new.loc[df_cik_ticker_new.CIK == 849636, 'Ticker'] = "RSPI"
df_cik_ticker_new.loc[df_cik_ticker_new.CIK == 855683, 'Ticker'] = "MLSS"
df_cik_ticker_new.loc[df_cik_ticker_new.CIK == 1104485, 'Ticker'] = "NOG"
df_cik_ticker_new.loc[df_cik_ticker_new.CIK == 708821, 'Ticker'] = "PAR"
df_cik_ticker_new.loc[df_cik_ticker_new.CIK == 921114, 'Ticker'] = "ARMP"

df_cik_ticker_new.to_csv(all_data_local_folder + "cik_ticker_corrected.csv", sep="|", header = True, index=False)

(13737, 8)


In [ ]:
df =  pd.read_csv(all_data_local_folder + "07_all_features_complete.csv", header = 0)
df = df[~df.ticker.isin(['ATLS','IFT'])]
df = df[~df.cik.isin([1117106,1308161])]

df.loc[df.cik == 1135185, 'ticker'] = "AAWW"
df.loc[df.cik == 849636,  'ticker'] = "RSPI"
df.loc[df.cik == 855683,  'ticker'] = "MLSS"
df.loc[df.cik == 1104485, 'ticker'] = "NOG"
df.loc[df.cik == 708821,  'ticker'] = "PAR"
df.loc[df.cik == 921114,  'ticker'] = "ARMP"

df = df.drop(['index', 'level_0'], axis=1)

df.to_csv(all_data_local_folder + "07_all_features_complete_corrected.csv", header = True, index=False)

## 00_Tools

In [ ]:
def load_data() -> pd.DataFrame:
    df = pd.read_csv(all_data_local_folder + "07_all_features_complete_corrected.csv", header=0)

    df.period = pd.to_datetime(df.period)
    df.filed = pd.to_datetime(df.filed)
    
    df.sort_values('period', inplace = True)
    df.reset_index(inplace = True)
    return df

In [ ]:
def load_cik_ticker() -> pd.DataFrame:
    df = pd.read_csv(all_data_local_folder + "cik_ticker_corrected.csv", sep="|", header = 0)
    return df

## 01_Load required ticker symbols

In [ ]:
tickers = list(load_data().ticker.sort_values().unique())
print(len(tickers))

3050


## 02_Prepare directory structure

In [ ]:
def ensure_dir(file_path:str):
    """ Creates an empty directory if it doesn't exist
    """
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        print(file_path)
        os.makedirs(directory)

In [ ]:
import string

def create_dir_structure(folder:str):
    """ Creates the folder and creates also sub-folders ranging from 'A' to 'Z' beneath it.
    """
    ensure_dir(folder)
    for char in string.ascii_uppercase:
        ensure_dir(stock_data_folder + char + "/")

In [ ]:
create_dir_structure(stock_data_folder)

## 03_Download historical data
Unfortuantely, it is not that easy to get historical data of of companies that are no longer listed or went out of business for what reason ever. So we will only be able to work with companies, which are still in business. That is a problem, since a serious "survivor" bias is introduced in the data. It would be important to also have data of "failed" companies.

We will be using two different libraries to download: yahoo-historical and yfinance. yfinance has the advantage that Open, High, Low and Close can be download "adjusted" meaning that splits and payed dividends are taken into consideration. The standard Open, High, Low and Close from yahoo is only adjusted for splits.

The code in a Jupyter Notebook cannot be parallelized that easily, therefore there is the "08_Download_StodkData.py" Script that implements a parallelized version.

In [ ]:
def download_and_store(ticker: str, downloader_function, filename_function):
    filename = filename_function(ticker)
    
    if os.path.isfile(filename):
        return

    print("loading: ", ticker)
    try:
        data = downloader_function(ticker, 2010)
    except Exception as ex:
        print("failed: ", ticker, " - ", str(ex))
        return

    if data.shape[0] > 200:
        df.to_csv(filename, sep=',', encoding='utf-8', index=True)
    else:
        print("too short: ", data.shape[0], " - ", ticker)

### functions to download using yahoo_historical package

In [ ]:
def downloadTicker_yahoo_historical(stock:str, startyear:int)->pd.DataFrame:
    """ Download data using yahoo_historical Fetcher class
    """
    stockFetcher = Fetcher(stock,[startyear,1,1],[2020,12,10])
    stock_df = stockFetcher.get_historical()
    return stock_df

def get_filename_yahoo_financials(ticker:str):
    return stock_data_folder + ticker[0] + "/" + ticker + ".csv"

def serial_download_using_yahoo_financials(tickers: List[str]):
    for ticker in tickers:
        download_and_store(ticker, downloadTicker_yahoo_historical, get_filename_yahoo_financials)

### functions to download using yfinance package

In [ ]:
def downloadTicker_yfinance(stock:str, startyear:int)->pd.DataFrame:
    """ Download data using yfinance package.
        it provides the major advantage, that the HOLC can be directly "adjusted", meaning they are adapted for splits and dividends.
    """
    stock_df = yf.download(stock,start=str(startyear)+"-01-01",end="2020-12-10", progress=False, auto_adjust=True)
    return stock_df

def get_filename_yfinance(ticker:str):
    return stock_data_folder + ticker[0] + "/" + ticker + "_2.csv"

def serial_download_using_yfinance(tickers: List[str]):
    for ticker in tickers:
        download_and_store(ticker, downloadTicker_yfinance, get_filename_yfinance)

### function to check how many historical data could be downloaded

In [ ]:
def count_and_return_missing(tickers:List[str], filename_function)-> List[str]:
    missing:List[str] = []
    counter_na = 0
    counter_ready = 0
    for ticker in tickers:
        if os.path.isfile(filename_function(ticker)) == False:
            counter_na += 1
            missing.append(ticker)
        else:
            counter_ready += 1
    print("count of missing:    ", counter_na)
    print("count of downloaded: ", counter_ready)
    return missing

### download using yahoo_historicals package

In [ ]:
serial_download_using_yahoo_financials(tickers)
count_and_return_missing(tickers, get_filename_yahoo_financials)

### download using yfinance package

In [ ]:
serial_download_using_yfinance(tickers)
count_and_return_missing(tickers, get_filename_yfinance)

skipping:  A
skipping:  AA
loading:  AAC

1 Failed download:
- AAC: No data found, symbol may be delisted
too short:  0  -  AAC
skipping:  AAL
skipping:  AAME
loading:  AAN
too short:  10  -  AAN
skipping:  AAOI
skipping:  AAON
skipping:  AAPL
skipping:  AAT
skipping:  AAWW
skipping:  AB
skipping:  ABAX
skipping:  ABBV
skipping:  ABC
skipping:  ABCB
skipping:  ABCD
skipping:  ABCO
loading:  ABFS

1 Failed download:
- ABFS: No data found for this date range, symbol may be delisted
too short:  0  -  ABFS
skipping:  ABG
skipping:  ABM
skipping:  ABMD
skipping:  ABR
skipping:  ABT
loading:  ABTL

1 Failed download:
- ABTL: No data found for this date range, symbol may be delisted
too short:  0  -  ABTL
skipping:  ACAD
loading:  ACAT

1 Failed download:
- ACAT: No data found for this date range, symbol may be delisted
too short:  0  -  ACAT
skipping:  ACC
skipping:  ACET
skipping:  ACFC
skipping:  ACFN
skipping:  ACGL
skipping:  ACHC
loading:  ACHN

1 Failed download:
- ACHN: No data found,

['AAC',
 'AAN',
 'ABFS',
 'ABTL',
 'ACAT',
 'ACHN',
 'ACPW',
 'ACXM',
 'ADAT',
 'ADGE',
 'ADK',
 'AEGR',
 'AEPI',
 'AETI',
 'AFOP',
 'AGII',
 'AH',
 'AHP',
 'AHS',
 'AIRM',
 'AKS',
 'ALJ',
 'ALN',
 'ALR',
 'ALXA',
 'AMB',
 'AMCC',
 'AMIC',
 'AMID',
 'AMRI',
 'AMSG',
 'ANAC',
 'ANAD',
 'ANX',
 'APC',
 'APNB',
 'APOL',
 'APU',
 'ARCI',
 'ARCP',
 'ARCX',
 'AREX',
 'ARG',
 'ARIA',
 'ARIS',
 'ARO',
 'ARQL',
 'ARSD',
 'ARTX',
 'ARVT',
 'ASBB',
 'ASBC',
 'ASEI',
 'ASHN',
 'ASTM',
 'ASXHTW',
 'ATML',
 'ATRM',
 'ATU',
 'AV',
 'AVCA',
 'AVP',
 'AVX',
 'AWLD',
 'AXLL',
 'BABY',
 'BAMM',
 'BBCN',
 'BBG',
 'BBT',
 'BCR',
 'BEAV',
 'BGF',
 'BHI',
 'BID',
 'BIOA',
 'BIOD',
 'BIOF',
 'BIOS',
 'BKJ',
 'BKMU',
 'BKS',
 'BLOX',
 'BLT',
 'BMOM',
 'BNCN',
 'BOBE',
 'BOFI',
 'BOJF',
 'BONE',
 'BONT',
 'BOTA',
 'BPI',
 'BRCD',
 'BRS',
 'BRSS',
 'BSFT',
 'BSIC',
 'BTX',
 'BVSN',
 'BVX',
 'BWC',
 'BWINB',
 'BWLD',
 'BWS',
 'BYLK',
 'CAB',
 'CACB',
 'CADC',
 'CAM',
 'CARB',
 'CAXG',
 'CBAK',
 'CBF',
 'CBG',
 'C

## 04_Download Additional Data
There are some important information which are not available from the SEC data. These are mainly the sector, industry the company is in, as well as the marketCap and the sharesOutstanding. These attributes are downloaded into a separate file.

The code in a Jupyter Notebook cannot be parallelized that easily, therefore there is the "08_Download_StodkData.py" Script that implements a parallelized version.

The data is downloaded from Yahoo Finance. However, Yahoo Finance restricts the number of calls that can be made from one computer within a certain amount of time. If there are too many requests from the same computer, yahoo answers with a HTTP Error 404.  Therefore the code has to be able to be started multiple times and has to keep track of the data that already has been downloaded successfully.

In the script "08_Download_StockData.py" there is a also a parallelized version of the code

In [ ]:
def get_add_data(ticker:str) -> Dict[str,str]:
    """
    get some additional info using yfinance libary.
    if too many requests are made during a certain time, yahoo finance will block and start to answer with a 404.
    so it is necessary to keep track of the sucessfully or missing entries
    """
    
    try:
        print(ticker)
        info = yf.Ticker(ticker)
        return {'ticker'            : ticker,
                'sector'            : info.info['sector'],
                'industry'          : info.info['industry'],
                'marketCap'         : info.info['marketCap'],
                'sharesOutstanding' : info.info['sharesOutstanding'],
                'message'           : None}
    
    except Exception as ex:
        msg = str(ex)
        if "404" in msg:
            return None

        # ticker information was not found on yahoo, very likely because the company is not listed anymore
        return {'ticker'            : ticker,
                'sector'            : None,
                'industry'          : None,
                'marketCap'         : None,
                'sharesOutstanding' : None,
                'message'           : msg}

In [ ]:
def download_add_info(tickers: List[str]):
    """ serial version of downloading the additional information.
    """
    
    start = time.time()
    # read the old data
    filename = all_data_local_folder + "08_add_ticker_info.csv"
    current = pd.read_csv(filename, sep=',', encoding='utf-8', header=0)

    # figure out, which ticker information hasn't been downloaded already
    current_tickers = current.ticker.to_list()
    tickers = list(set(tickers) - set(current_tickers))
    
    # create now
    data = pd.DataFrame(columns=['ticker', 'sector', 'industry','marketCap', 'sharesOutstanding','message'],)

    list_infos = []
    for ticker in tickers:
        list_infos.append(get_add_data(ticker))

    entries = [x for x in list_infos if x is not None]
    print("new entries: ", len(entries))

    for entry in entries:
        data = data.append(entry, ignore_index=True)

    data = pd.concat([current, data])

    print(data.shape)
    print("duration: ", time.time() - start)

    data.to_csv(all_data_local_folder + "08_add_ticker_info.csv", sep=',', encoding='utf-8', index=False)

In [ ]:
download_add_info(tickers)

new entries:  0
(3053, 6)
duration:  0.049045562744140625


## 05_Normalization and feature generation of historical data
In order to be able to compare the historical stock price data of different companies, they have to be normalized. 
- First, Open, High, and Low are compared with the Close price.
- Second, the Close price is normalized (divided by) the last Close Price
- Third, the volume is normalized by that sharesOutstanding 

Furthermore, as additional features, well we add
- the percentual change of the close Price, compared to the day before
- the percentual change of the traded volume, compared to the day before
- day of week, day of month, day of year, and month of year

And again, there is also a parallelized version of this code in "08_Download_StockData.py"

### reading the data

In [ ]:
# reading all files with adjusted OHLC Data (stored stored in "*_2.csv" files)
import glob

def get_data_files() -> List[str]:
    return glob.glob(stock_data_folder + "**/*_2.csv",recursive = True)

In [ ]:
def read_stockdata(filename: str) -> pd.DataFrame:
    df= pd.read_csv(filename, header = 0, sep=",", encoding='utf-8')
    df.Date = pd.to_datetime(df.Date)
    return df

In [ ]:
def read_additional_info() -> pd.DataFrame:
    return pd.read_csv(all_data_local_folder + "add_ticker_info.csv", sep=',', encoding='utf-8', header=0)

In [ ]:
def get_ticker_from_filename(filename: str) -> str:
    return filename[len(stock_data_folder) + 2 : -6]

### normalization

In [ ]:
def norm_historical_data(df: pd.DataFrame, shares_outstanding: float):
    lastClose = df[-1:].Close.values[0]
    df['close_norm'] = df.Close / lastClose
    df['high_norm']  = (df.High - df.Close) / df.Close
    df['low_norm']   = (df.Low - df.Close) / df.Close
    df['open_norm']  = (df.Open -df.Close) / df.Close
    df['Volume']     = df['Volume'].astype(float)
    df['volume_norm']= df.Volume / shares_outstanding

### additional features

In [ ]:
def create_features_from_historical_data(df: pd.DataFrame):
    
        df['close_chg'] = 0.0
        
        close_data = df.Close.to_numpy()
        close_change = ((close_data[1:] - close_data[:-1]) / close_data[:-1])    
    
        df.loc[df.index[1:],'close_chg'] = close_change
        
        
        df['volume_chg'] = 0.0
        
        volume_data = df.Volume.to_numpy()
        volume_change = ((volume_data[1:] - volume_data[:-1]) / volume_data[:-1])    
    
        df.loc[df.index[1:],'volume_chg'] = volume_change
        
        df['day_of_week']   = df.Date.dt.dayofweek / 7
        df['day_of_month']  = df.Date.dt.day / 31
        df['day_of_year']   = df.Date.dt.dayofyear / 365
        df['week_of_year']  = df.Date.dt.weekofyear / 52
        df['month_of_year'] = df.Date.dt.month / 12

### process stocks

In [ ]:
def process_stock_data(filename:str, df_add_info: pd.DataFrame):
    ticker = get_ticker_from_filename(filename)
    new_file = stock_data_folder + ticker[0] + "/" + ticker + "_processed.csv"
    
    if os.path.isfile(new_file):
        return
    
    print("process: ", ticker)

    df_stock = read_stockdata(filename)
    df_stock_add_info = df_add_info[df_add_info.ticker == ticker]
    
    sharesOutstanding = df_stock_add_info[-1:].sharesOutstanding.values[0]
    norm_historical_data(df_stock, sharesOutstanding)
    create_features_from_historical_data(df_stock)
    
    df_stock.to_csv(new_file, sep=',', encoding='utf-8', index=False)

In [ ]:
process_stock_data(stock_data_folder + "A/AAPL_2.csv", df_add_info) # test with one stock

In [ ]:
df_add_info = read_additional_info()
files = get_data_files()
for filename in files:
    process_stock_data(filename, df_add_info)

## XX_Trials

In [ ]:
tickers

array(['A', 'AA', 'AAC', ..., 'ZTM', 'ZTS', 'ZUMZ'], dtype=object)

In [ ]:
cik_tick_df = load_cik_ticker()
cik_tick_df.columns

Index(['CIK', 'Ticker', 'Name', 'Exchange', 'SIC', 'Business', 'Incorporated',
       'IRS'],
      dtype='object')

In [ ]:
cik_tick_df[cik_tick_df.Ticker == 'CNAT']

,CIK,Ticker,Name,Exchange,SIC,Business,Incorporated,IRS
2685,1383701,CNAT,Conatus Pharmaceuticals Inc,NASDAQ,2834.0,CA,DE,203183915.0


In [ ]:
stockFetcher = Fetcher('AAPL',[2010,1,1])
stock_df = stockFetcher.get_historical()

In [ ]:
stock_df.shape

(2756, 7)

In [ ]:
stock_df[:10]

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,26.000362,26.177889,25.870815,26.129908,20.155407,10829000
1,2010-01-05,26.134706,26.134706,25.789249,25.918797,19.992567,10562100
2,2010-01-06,25.880411,26.096321,25.837231,26.062737,20.103596,11401400
3,2010-01-07,26.057938,26.283443,25.942785,26.278646,20.270136,12857200
4,2010-01-08,26.273848,26.508949,26.235464,26.412991,20.373764,12148600
5,2010-01-11,26.532940,26.619305,26.422586,26.547335,20.477396,10947200
6,2010-01-12,26.484961,26.652889,26.403395,26.470566,20.418177,10939300
7,2010-01-13,26.321829,26.648092,26.317030,26.528143,20.612034,10538900
8,2010-01-14,26.494556,26.686476,26.389000,26.556931,20.634401,8982400
9,2010-01-15,26.561728,26.662485,26.465769,26.624102,20.686594,16132500


https://aroussi.com/post/python-yahoo-finance

In [ ]:
import yfinance as yf
msft = yf.Ticker("AAPL")
print(msft.info)

{'zip': '95014', 'sector': 'Technology', 'fullTimeEmployees': 147000, 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radi

In [ ]:
msft.get_balance_sheet()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
print(msft.info['sector'])
print(msft.info['industry'])
print(msft.info['marketCap'])
print(msft.info['sharesOutstanding'])


Technology
Software—Infrastructure
1612352192512
7560500224


In [ ]:

data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30", )


[*********************100%***********************]  2 of 2 completed


In [ ]:
data

Adj Close                  Close                   High  \
                 AAPL         SPY       AAPL         SPY       AAPL   
Date                                                                  
2017-01-03  27.277639  209.785889  29.037500  225.240005  29.082500   
2017-01-04  27.247108  211.033981  29.004999  226.580002  29.127501   
2017-01-05  27.385668  210.866318  29.152500  226.399994  29.215000   
2017-01-06  27.690971  211.620728  29.477501  227.210007  29.540001   
2017-01-09  27.944603  210.922211  29.747499  226.460007  29.857500   
...               ...         ...        ...         ...        ...   
2017-04-24  33.879871  221.858353  35.910000  237.169998  35.987499   
2017-04-25  34.089794  223.149246  36.132500  238.550003  36.224998   
2017-04-26  33.889301  223.008942  35.919998  238.399994  36.150002   
2017-04-27  33.915257  223.196014  35.947498  238.600006  36.040001   
2017-04-28  33.882225  222.709595  35.912498  238.080002  36.075001   

                              Low                   Open              \
                   SPY       AAPL         SPY       AAPL         SPY   
Date                                                                   
2017-01-03  225.830002  28.690001  223.880005  28.950001  225.039993   
2017-01-04  226.750000  28.937500  225.610001  28.962500  225.619995   
2017-01-05  226.580002  28.952499  225.479996  28.980000  226.270004   
2017-01-06  227.750000  29.117500  225.899994  29.195000  226.529999   
2017-01-09  227.070007  29.485001  226.419998  29.487499  226.910004   
...                ...        ...         ...        ...         ...   
2017-04-24  237.410004  35.794998  234.559998  35.875000  237.179993   
2017-04-25  238.949997  35.967499  237.809998  35.977501  237.910004   
2017-04-26  239.529999  35.845001  238.350006  36.117500  238.509995   
2017-04-27  238.949997  35.827499  237.979996  35.980000  238.770004   
2017-04-28  238.929993  35.817501  237.929993  36.022499  238.899994   

               Volume             
                 AAPL        SPY  
Date                              
2017-01-03  115127600   91366500  
2017-01-04   84472400   78744400  
2017-01-05   88774400   78379000  
2017-01-06  127007600   71559900  
2017-01-09  134247600   46939700  
...               ...        ...  
2017-04-24   68537200  119209900  
2017-04-25   75486000   76698300  
2017-04-26   80164800   84702500  
2017-04-27   56985200   57410300  
2017-04-28   83441600   63532800  

[81 rows x 12 columns]

In [ ]:
data = yf.download("AAC", start="2010-01-01", end="2020-12-11")
data

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC: No data found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
aacob = yf.Ticker("AAPL")
aacob.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 147000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

quotes for delisted companies

https://www.quantshare.com/item-81-historical-quotes-for-delisted-us-stocks
https://money.stackexchange.com/questions/18723/how-to-find-historical-stock-price-for-a-de-listed-or-defunct-company